In [29]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import PCA

In [2]:
data = pd.read_csv('zoo.csv')

In [3]:
data.head()

animal name  hair  feathers  eggs  milk  airborne  aquatic  predator  \
0    aardvark     1         0     0     1         0        0         1   
1    antelope     1         0     0     1         0        0         0   
2        bass     0         0     1     0         0        1         1   
3        bear     1         0     0     1         0        0         1   
4        boar     1         0     0     1         0        0         1   

   toothed  backbone  breathes  venomous  fins  legs  tail  domestic  catsize  \
0        1         1         1         0     0     4     0         0        1   
1        1         1         1         0     0     4     1         0        1   
2        1         1         0         0     1     0     1         0        0   
3        1         1         1         0     0     4     0         0        1   
4        1         1         1         0     0     4     1         0        1   

   type  
0     1  
1     1  
2     4  
3     1  
4     1

In [4]:
data.describe()

hair    feathers        eggs        milk    airborne     aquatic  \
count  101.000000  101.000000  101.000000  101.000000  101.000000  101.000000   
mean     0.425743    0.198020    0.584158    0.405941    0.237624    0.356436   
std      0.496921    0.400495    0.495325    0.493522    0.427750    0.481335   
min      0.000000    0.000000    0.000000    0.000000    0.000000    0.000000   
25%      0.000000    0.000000    0.000000    0.000000    0.000000    0.000000   
50%      0.000000    0.000000    1.000000    0.000000    0.000000    0.000000   
75%      1.000000    0.000000    1.000000    1.000000    0.000000    1.000000   
max      1.000000    1.000000    1.000000    1.000000    1.000000    1.000000   

         predator     toothed    backbone    breathes    venomous        fins  \
count  101.000000  101.000000  101.000000  101.000000  101.000000  101.000000   
mean     0.554455    0.603960    0.821782    0.792079    0.079208    0.168317   
std      0.499505    0.491512    0.384605    0.407844    0.271410    0.376013   
min      0.000000    0.000000    0.000000    0.000000    0.000000    0.000000   
25%      0.000000    0.000000    1.000000    1.000000    0.000000    0.000000   
50%      1.000000    1.000000    1.000000    1.000000    0.000000    0.000000   
75%      1.000000    1.000000    1.000000    1.000000    0.000000    0.000000   
max      1.000000    1.000000    1.000000    1.000000    1.000000    1.000000   

             legs        tail    domestic     catsize        type  
count  101.000000  101.000000  101.000000  101.000000  101.000000  
mean     2.841584    0.742574    0.128713    0.435644    2.831683  
std      2.033385    0.439397    0.336552    0.498314    2.102709  
min      0.000000    0.000000    0.000000    0.000000    1.000000  
25%      2.000000    0.000000    0.000000    0.000000    1.000000  
50%      4.000000    1.000000    0.000000    0.000000    2.000000  
75%      4.000000    1.000000    0.000000    1.000000    4.000000  
max      8.000000    1.000000    1.000000    1.000000    7.000000

In [5]:
# Encoding the target variable 'type' if it is a categorical variable
label_encoder = LabelEncoder()
data['type'] = label_encoder.fit_transform(data['type'])

In [6]:
# Check for missing values
print(data.isnull().sum())

animal name    0
hair           0
feathers       0
eggs           0
milk           0
airborne       0
aquatic        0
predator       0
toothed        0
backbone       0
breathes       0
venomous       0
fins           0
legs           0
tail           0
domestic       0
catsize        0
type           0
dtype: int64


In [7]:
# Boxplot to visualize outliers
plt.figure(figsize=(12, 8))
sns.boxplot(data=data)
plt.show()

In [8]:
# Remove outliers if necessary (example: using IQR method)
Q1 = data.quantile(0.25)
Q3 = data.quantile(0.75)
IQR = Q3 - Q1

C:\Users\vinit\AppData\Local\Temp\ipykernel_16140\583114254.py:2: FutureWarning: The default value of numeric_only in DataFrame.quantile is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  Q1 = data.quantile(0.25)
C:\Users\vinit\AppData\Local\Temp\ipykernel_16140\583114254.py:3: FutureWarning: The default value of numeric_only in DataFrame.quantile is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  Q3 = data.quantile(0.75)


In [9]:
# Assuming outliers are data points below Q1 - 1.5*IQR or above Q3 + 1.5*IQR
data = data[~((data < (Q1 - 1.5 * IQR)) |(data > (Q3 + 1.5 * IQR))).any(axis=1)]

C:\Users\vinit\AppData\Local\Temp\ipykernel_16140\3855852373.py:2: FutureWarning: Automatic reindexing on DataFrame vs Series comparisons is deprecated and will raise ValueError in a future version. Do `left, right = left.align(right, axis=1, copy=False)` before e.g. `left == right`
  data = data[~((data < (Q1 - 1.5 * IQR)) |(data > (Q3 + 1.5 * IQR))).any(axis=1)]


In [10]:
# Splitting the dataset into features and target variable
X = data.drop(columns=['type'])
y = data['type']

In [12]:
# Encode categorical features using one-hot encoding
X = pd.get_dummies(X, drop_first=True)

In [13]:
# Standardize the features
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [30]:
# Apply PCA to reduce dimensions to 2 for visualization purposes
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X)

In [14]:
# Split dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [15]:
# Initialize KNN classifier
knn = KNeighborsClassifier()

In [16]:
# Define parameter grid
param_grid = {
    'n_neighbors': range(1, 21),
    'p': [1, 2]  # p=1 for Manhattan, p=2 for Euclidean
}

In [25]:
# Initialize GridSearchCV
grid_search = GridSearchCV(KNeighborsClassifier(), param_grid, cv=5)
grid_search.fit(X_train, y_train)

C:\Users\vinit\anaconda3\Lib\site-packages\sklearn\model_selection\_split.py:776: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(


GridSearchCV(cv=5, estimator=KNeighborsClassifier(),
             param_grid={'n_neighbors': range(1, 21), 'p': [1, 2]})

In [18]:
# Best parameters
print("Best parameters:", grid_search.best_params_)

Best parameters: {'n_neighbors': 1, 'p': 1}


In [26]:
# Use the best parameters to fit the model
knn = KNeighborsClassifier(n_neighbors=grid_search.best_params_['n_neighbors'], p=grid_search.best_params_['p'])
knn.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=1, p=1)

In [20]:
# Predict on test set
y_pred = knn.predict(X_test)

In [21]:
# Evaluation metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")

Accuracy: 1.0
Precision: 1.0
Recall: 1.0
F1 Score: 1.0


In [31]:
# Visualizing decision boundaries (for a 2D subset of the features)
knn_pca = KNeighborsClassifier(n_neighbors=grid_search.best_params_['n_neighbors'], p=grid_search.best_params_['p'])
knn_pca.fit(X_pca, y)

KNeighborsClassifier(n_neighbors=1, p=1)

In [32]:
# Create a mesh grid
x_min, x_max = X_pca[:, 0].min() - 1, X_pca[:, 0].max() + 1
y_min, y_max = X_pca[:, 1].min() - 1, X_pca[:, 1].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.02),
                     np.arange(y_min, y_max, 0.02))

In [33]:
# Predict on mesh grid
Z = knn_pca.predict(np.c_[xx.ravel(), yy.ravel()])
Z = Z.reshape(xx.shape)

In [34]:
# Plot decision boundary
plt.contourf(xx, yy, Z, alpha=0.8)
plt.scatter(X_pca[:, 0], X_pca[:, 1], c=y, edgecolor='k', marker='o')
plt.xlabel('PCA Feature 1')
plt.ylabel('PCA Feature 2')
plt.title('KNN Decision Boundaries after PCA')
plt.show()

In [ ]:
"""What are the key hyperparameters in KNN?
   1: Number of Neighbors (n_neighbors): This is the number of nearest neighbors to consider when making a prediction. 
   2:Distance Metric (metric): This defines how the distance between two points is calculated. The most common distance 
     metrics are:

        Euclidean Distance (metric='euclidean'): The straight-line distance between two points in Euclidean space.
        Manhattan Distance (metric='manhattan'): The sum of the absolute differences of their coordinates.
        Minkowski Distance (metric='minkowski'): A generalization of Euclidean and Manhattan distances. The parameter p 
        determines which specific distance metric is used.
    3:Power Parameter for Minkowski Distance (p): This parameter is used when metric='minkowski'. 
    4: Weight Function (weights): This determines the weight assigned to the neighbors' contributions when making predictions:

        Uniform (weights='uniform'): All neighbors have equal weight.
        Distance (weights='distance'): Closer neighbors have a greater influence on the prediction."""

In [ ]:
"""What distance metrics can be used in KNN?
     Distance Metric (metric): This defines how the distance between two points is calculated. The most common distance 
     metrics are:

        Euclidean Distance (metric='euclidean'): The straight-line distance between two points in Euclidean space.
        Manhattan Distance (metric='manhattan'): The sum of the absolute differences of their coordinates.
        Minkowski Distance (metric='minkowski'): A generalization of Euclidean and Manhattan distances. The parameter p 
         determines which specific distance metric is used."""